# To Do List

#### step1. dictionary 생성
> step1-1. makers별 foodid들이 들어있는 dict

> step1-2. foodid별 foodname이 들어있는 dict

> step1-3. group별 user들이 있는 dict

> step1-4. group별 최근 이용한 N개의 makers가 있는 dict

> step1-5. user별 최근 주문한 N개의 foodid가 있는 dict

---
Cosine Similarity : makersid-makersid, foodid-foodid

---

#### step2. CB추천 완성
> 필요한 DF 생성

#### EXTRA Exercise.
> group이 들어왔을 때, 가장 최근에 시킨 메이커스 기반 메이커스 추천

> group의 user들에게 해당 메이커스에 있는 음식추천

##### input & output
>input: groupid

>output: 그룹에게 TopN개 추천 recomm_makers, makers_score

>output: 유저에게 TopN개 추천 recomm_food, food_score

In [1]:
# 필요 패키지 import
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity

### step1-1. maker별 foodid {makersid : {foodid set}}

In [2]:
makers_food = pd.read_csv('data/back/food_makers.csv')

makers_foods_dic = {}
for makersid in makers_food['MakersId'].unique():
    makers_foods_dic[makersid] = set({})

for makersid, foodid in zip(makers_food['MakersId'], makers_food['FoodId'].values):
    makers_foods_dic[makersid].add(int(foodid))
makers_foods_dic

{28: {18,
  71,
  73,
  74,
  115,
  116,
  134,
  252,
  409,
  502,
  503,
  864,
  865,
  866,
  867,
  909,
  1044,
  1201,
  1294,
  1365,
  1695,
  1932,
  1947,
  2599,
  2601,
  2602,
  2603,
  2997,
  3069,
  3150,
  3404,
  3405,
  3406},
 35: {27, 28, 51, 52, 234},
 42: {48,
  49,
  50,
  93,
  151,
  196,
  197,
  885,
  886,
  944,
  945,
  946,
  989,
  1617,
  1678,
  1680},
 41: {42,
  43,
  44,
  45,
  46,
  47,
  94,
  117,
  118,
  119,
  169,
  170,
  171,
  178,
  179,
  205,
  206,
  235,
  323,
  348,
  363,
  371,
  373,
  404,
  405,
  410,
  412,
  416,
  417,
  418,
  482,
  483,
  561,
  563,
  564,
  565,
  566,
  586,
  587,
  590,
  591,
  912,
  964,
  1027,
  1163,
  1164,
  1196,
  1197,
  1275,
  1396,
  1397,
  1401,
  1402,
  1404,
  1616,
  1618,
  1682,
  2076,
  2249,
  2373,
  2374,
  2375,
  2376,
  2377,
  2397,
  2435,
  2436,
  2437,
  2492,
  2493,
  2494,
  2495,
  2496,
  2497,
  2498,
  2499,
  2500,
  2501,
  2506,
  2510,
  2518,
  251

### step1-2. foodid별 foodname {foodid : foodname}

In [3]:
food_name_dic = {}

for foodid in makers_food['FoodId'].unique():
    food_name_dic[foodid] = ""

for foodid, foodname in zip(makers_food['FoodId'],makers_food['FoodName']):
    food_name_dic[foodid] = foodname
food_name_dic

{18: '(오픈특가)차돌구이도시락*',
 51: '1인세트(달리)*',
 50: '꿍 카오팟(달리)*',
 43: '돈카츠김치찌개(달리)*',
 42: '돼지반김치짜글이(달리)*',
 44: '연어장덮밥(달리)*',
 47: '돈카츠커리(달리)*',
 39: '연어회덮밥(달리)*',
 41: '해초샐러드비빔밥(달리)*',
 49: '뿌팟퐁커리(달리)*',
 48: '똠얌꿍누들(달리)*',
 91: '(프로모션)우렁강된장&직화구이쌈밥*',
 24: '(프로모션)달래장 봄나물 비빔밥*',
 52: '2인세트(달리)*',
 95: '한우국밥*',
 20: '응원한닭 안심샐러드*',
 22: '회오리오무돈까스*',
 23: '(프로모션)쇠고기야채죽*',
 108: '(클로)수제함박스테이크*',
 106: '(클로)비프스테이크박스*',
 32: '(프로모션)수란스파이시포크라이스*',
 31: '(프로모션)통살치킨마요*',
 40: '참치회덮밥(달리)*',
 30: '춘천식닭갈비도시락(8구)*',
 46: '수제등심돈카츠(달리)*',
 107: '(클로)마포식돼지구이도시락8구*',
 33: '목살스테이크*',
 110: '폭립도시락*',
 70: '탕볶밥*',
 97: '연어회덮밥*',
 134: '(오픈특가)LA갈비도시락*',
 99: '해초샐러드덮밥*',
 98: '참치회덮밥*',
 117: '(클로)송고버섯소불고기덮밥*',
 45: '하카다 카레라이스(달리)*',
 132: '(클로)제육백반*',
 112: '(클로)1인보쌈도시락(M)*',
 116: '오징어볶음도시락*',
 118: '(클로)쿵파오데리야끼치킨도시락*',
 67: '소고기국밥*',
 29: '깍두기김치볶음밥*',
 119: '(클로)연어장덮밥*',
 103: '폭립덮밥*',
 115: '갈비탕도시락*',
 96: '수제함박스테이크*',
 147: '후레쉬빌정식*',
 83: '큐브스테이크덮밥*',
 113: '(클로)1인보쌈도시락(L)*',
 25: '목살슬라이스덮밥*',
 93: '뿌팟퐁커리*'

### step1-3. group별 유저들 {groupid : {user set}}

In [4]:
group_users = pd.read_csv('data/order_food_data.csv')

group_users_dic = {}
for groupid in group_users['GroupId'].unique():
    group_users_dic[groupid] = set({})

for makersid, userid in zip(group_users['GroupId'], group_users['UserId'].values):
   group_users_dic[makersid].add(int(userid))
group_users_dic

{0: {3,
  7,
  17,
  21,
  23,
  39,
  136,
  198,
  317,
  319,
  324,
  331,
  333,
  343,
  344,
  366,
  375,
  426,
  449,
  451,
  591,
  592,
  593,
  599,
  605,
  615,
  720,
  778,
  794,
  795,
  803,
  812,
  874,
  932,
  939,
  942,
  967,
  977,
  995,
  1004,
  1017,
  1095,
  1122,
  1171,
  1173,
  1174,
  1183,
  1187,
  1190,
  1191,
  1210,
  1232,
  1274,
  1329,
  1359,
  1361,
  1365,
  1366,
  1382,
  1417,
  1420,
  1462,
  1484,
  1514,
  1524,
  1546,
  1568,
  1571,
  1575,
  1577,
  1593,
  1600,
  1602,
  1653,
  1662,
  1683,
  1835,
  1869,
  1880,
  1892,
  1893,
  1973,
  1974,
  1980,
  1983,
  1984,
  2006,
  2010,
  2116,
  2151,
  2152,
  2165,
  2191,
  2194,
  2202,
  2204,
  2207,
  2220,
  2236,
  2237,
  2241,
  2246,
  2257,
  2271,
  2288,
  2305,
  2309,
  2327,
  2329,
  2337,
  2386,
  2430,
  2431,
  2450,
  2467,
  2493,
  2495,
  2519,
  2525,
  2526,
  2588,
  2591,
  2618,
  2619,
  2623,
  2642,
  2644,
  2664,
  2690,
  2693,
  27

### step1-4. group별 최근  이용한 N개의 makers가 있는 dict (1<=N<=5) 여기서 N은 5

In [5]:
# group별 최근 이용한 n개의 makersid dict 만들기
group_makers_df = pd.read_csv('data/order_food_data.csv')

x = group_makers_df[['OrderDate','GroupId', 'MakersId']]
x.sort_values(['OrderDate','GroupId'], ascending=False, inplace=True, ignore_index=True)

recently_n_makers_dic = {}
for groupid in group_users['GroupId'].unique():
    recently_n_makers_dic[groupid] = set({})

n = 5 # n은 5이하로. 즉, 최근 이용한 N개의 makersid
for g_id, m_id in zip(x['GroupId'].values, x['MakersId'].values):
    if len(recently_n_makers_dic[g_id]) != n:
        recently_n_makers_dic[g_id].add(m_id)
    else:
        pass
recently_n_makers_dic

/var/folders/8p/0rn2lvdj485dhnwbsnpgnb380000gp/T/ipykernel_25240/2155030670.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.sort_values(['OrderDate','GroupId'], ascending=False, inplace=True, ignore_index=True)


{0: {17, 92, 202, 230, 250},
 1: {25, 69, 81, 92, 136},
 12: {15, 28, 29, 41, 45},
 13: {15, 28, 29, 41},
 49: {16, 52, 60, 69, 107},
 20: {165, 170, 179, 199, 208},
 6: {51, 52, 67, 89, 98},
 10: {16, 30, 158, 228, 245},
 39: {37, 116, 179, 180, 198},
 2: {165, 180, 187, 240, 247},
 31: {28, 51, 85, 92, 103},
 59: {17, 22, 37, 136, 158},
 3: {37, 60, 116, 237, 246},
 129: {28, 158, 174, 225, 246},
 132: {28, 158, 165, 173, 200},
 139: {28, 158, 165, 174, 175},
 60: {30, 116, 129, 184, 228},
 43: {81, 137, 180, 184, 186},
 133: {22, 28, 147, 169, 180},
 148: {28, 158, 172, 173, 174},
 151: {28, 158, 172, 173, 174},
 160: {28, 174},
 143: {28, 158, 174, 225, 246},
 156: {28, 158, 174, 225, 246},
 161: {28, 173, 174},
 162: {28, 158, 174, 200, 225},
 172: {28, 158, 174, 225, 246},
 4: {29},
 9: {22, 28, 30, 45, 54},
 7: {30, 37, 111, 137, 164},
 26: {24, 30, 51, 93, 110},
 16: {1, 25, 29, 54, 62},
 8: {16, 17, 165, 184, 249},
 5: {17, 29, 45, 55, 62},
 11: {52},
 19: {17, 38, 70, 151, 15

### step1-5. user별 최근 주문한 N개의 foodid가 있는 dict (1<=N<=5) 여기서 N은 5

In [6]:
# user별 최근 이용한 n개의 foodid dict 만들기
food_df = pd.read_csv('data/order_food_data.csv')

x = food_df[['OrderDate','UserId', 'FoodId']]
x.sort_values(['OrderDate','UserId'], ascending=False, inplace=True, ignore_index=True)

recently_n_food_dic = {}
for foodid in food_df['UserId'].unique():
    recently_n_food_dic[foodid] = set({})

n = 5 # n은 5이하로. 즉, 최근 먹은 N개의 foodid
for u_id, f_id in zip(x['UserId'].values, x['FoodId'].values):
    if len(recently_n_food_dic[u_id]) != n:
        recently_n_food_dic[u_id].add(f_id)
    else:
        pass
recently_n_food_dic

/var/folders/8p/0rn2lvdj485dhnwbsnpgnb380000gp/T/ipykernel_25240/3861140545.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.sort_values(['OrderDate','UserId'], ascending=False, inplace=True, ignore_index=True)


{3: {18, 86},
 331: {179, 195, 213, 358, 512},
 347: {18, 31, 99, 110, 245},
 352: {813, 1482, 1890, 2660, 3412},
 381: {33, 93, 183, 2664, 3174},
 387: {32, 815, 1624, 2477, 3006},
 389: {937, 2465, 2730, 3258, 3404},
 351: {83, 97, 110, 136, 246},
 386: {1492, 2728, 3008, 3257, 3404},
 401: {31, 276, 282, 1482, 2662},
 399: {136, 171, 183, 241, 247},
 348: {69, 156, 277, 1492, 2662},
 353: {54, 59, 170, 247, 283},
 357: {1890, 2666, 2730, 3004, 3047},
 350: {937, 2462, 2659, 3035, 3317},
 7: {867, 959, 1452, 1478, 1684},
 388: {31, 240, 276, 278, 282},
 454: {56, 59, 103, 170, 276},
 448: {1482, 1683, 2465, 3035, 3337},
 449: {18, 22, 33, 135, 165},
 400: {827, 973, 1032, 2405, 2563},
 506: {69, 815, 867, 937, 2668},
 398: {939, 2451, 2666, 3008, 3173},
 508: {18},
 551: {18, 91, 101, 155},
 552: {18},
 553: {24, 91, 101, 103, 170},
 554: {18, 24, 101, 103, 170},
 556: {18, 91, 102, 170},
 555: {18, 91, 102, 170},
 317: {18, 958, 1959, 2095, 2196},
 319: {18, 848, 940, 1340, 1452},
 

### Makers Cosine Similarity

In [7]:
# makers cosine smilarity 구하기
mk_df = pd.read_csv('data/back/mk_df.csv')

mk_df.set_index('MakersId',inplace=True)
makers_cs_df = cosine_similarity(mk_df, mk_df)
makers_cs_df = pd.DataFrame(makers_cs_df, index = mk_df.index, columns= mk_df.index)
makers_cs_df

MakersId,28,35,42,41,43,29,17,1,38,37,...,245,247,238,249,250,251,252,257,253,254
MakersId,,,,,,,,,,,,,,,,,,,,,
28,1.000000,0.0,0.043482,0.697984,0.0,0.784856,0.663621,0.295918,0.771452,0.439039,...,0.583210,0.530937,0.228964,0.523919,0.332266,0.067351,0.801825,0.336940,0.572808,0.569275
35,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
42,0.043482,0.0,1.000000,0.047454,0.0,0.043533,0.048034,0.026496,0.047502,0.024360,...,0.027868,0.053659,0.010967,0.050181,0.032393,0.003681,0.045944,0.043006,0.043510,0.014073
41,0.697984,0.0,0.047454,1.000000,0.0,0.660110,0.767185,0.342195,0.638307,0.459853,...,0.430810,0.612665,0.254619,0.512535,0.369189,0.073972,0.732154,0.681555,0.842964,0.367079
43,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0.067351,0.0,0.003681,0.073972,0.0,0.236863,0.371590,0.427659,0.067502,0.455728,...,0.077198,0.441726,0.489667,0.041310,0.416667,1.000000,0.082675,0.064906,0.085872,0.035400
252,0.801825,0.0,0.045944,0.732154,0.0,0.861263,0.794449,0.348470,0.867867,0.496370,...,0.745002,0.685625,0.350939,0.598221,0.467778,0.082675,1.000000,0.465808,0.706277,0.542657
257,0.336940,0.0,0.043006,0.681555,0.0,0.416288,0.560401,0.223862,0.349442,0.270688,...,0.194375,0.430058,0.265896,0.434359,0.314792,0.064906,0.465808,1.000000,0.759316,0.248145


### Food Cosine Similarity
#### 487행까지는 foodtag가 없어서 다 0임. 이걸 없애줘야할까..?

In [8]:
food_cs_df = pd.read_csv('data/similarity/food_cs_df.csv', index_col=0)
food_cs_df

,18,51,50,43,42,44,47,39,41,49,...,10068,9812,10794,10772,7729,9941,7731,10453,10773,10633
FoodId,,,,,,,,,,,,,,,,,,,,,
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.276026,0.298142,0.389249,0.389249,1.000000,1.000000,0.866667,0.000000,0.389249,0.155700
7731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.276026,0.447214,0.389249,0.544949,0.866667,0.866667,1.000000,0.000000,0.544949,0.155700
10453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.144338,0.000000,0.150756,0.000000,0.000000,0.000000,1.000000,0.150756,0.000000


#### 아래가 다 0인 거 없애주는 코드인데
#### 하면 keyerror가 발생 나중에 user에게 food 추천할 때! 그래서 그냥 두는 게 나을수도 어차피 영향을 주지 않기 때문에.

In [9]:
# food_cs_df = food_cs_df.iloc[487:,:]
# food_cs_df

### step2. CB추천 완성

> GroupId, MakersId, Makers_Score DF 만들기

In [10]:
# 그룹을 담고 있는 set
group = set()
for g in group_users_dic:
    group.add(g)

In [11]:
i = 0
#행은 Makersid, 열은 GroupId
group_makers_recom_df = pd.DataFrame(index=[g for g in group_users_dic], columns=[m for m in makers_foods_dic])

# 그룹이 최근 이용한 N개의 메이커스 기반 메이커스 추천 #
def by_n_recently_used(makersid):
    global group_makers_recom_df
    cnt = 0
    # 주어진 메이커스와 다른 메이커스의 similarity를 가져온다
    sim_scores = makers_cs_df.loc[[m for m in makersid]].apply(lambda x: sum(x), axis = 0)
    sim_scores = sim_scores.apply(lambda x : x/len(makersid))
    
    for m_score in sim_scores.values:
        group_makers_recom_df.iloc[i,cnt] = m_score
        cnt += 1

# 위에서 만든 group_makers_recom_df에 value값 저장해주는 함수
for g in tqdm(group):
    by_n_recently_used(recently_n_makers_dic[g])
    i += 1

100%|██████████| 123/123 [00:00<00:00, 184.41it/s]


In [13]:
group_makers_recom_df.to_csv('data/recom_df/group_makers_recom_df.csv')

In [14]:
            # 열은 Makersid
#행은 GroupId
group_makers_recom_score = pd.read_csv('data/recom_df/group_makers_recom_df.csv')
group_makers_recom_score.rename(columns={'Unnamed: 0' : 'Groupid'}, inplace= True)
group_makers_recom_score.set_index('Groupid', inplace = True)
group_makers_recom_score

,28,35,42,41,43,29,17,1,38,37,...,245,247,238,249,250,251,252,257,253,254
Groupid,,,,,,,,,,,,,,,,,,,,,
0,0.277534,0.0,0.415492,0.307205,0.0,0.370526,0.463701,0.419611,0.309776,0.368289,...,0.252195,0.380411,0.429729,0.253227,0.453771,0.271806,0.359130,0.251432,0.328220,0.234685
1,0.342919,0.0,0.423882,0.396400,0.0,0.401002,0.437030,0.244177,0.378352,0.270964,...,0.277649,0.369377,0.236686,0.354388,0.271558,0.099024,0.432770,0.322014,0.389899,0.227301
12,0.527849,0.0,0.042915,0.595051,0.0,0.571257,0.654960,0.413089,0.608693,0.464638,...,0.408264,0.689179,0.368355,0.562968,0.450634,0.222711,0.648544,0.411468,0.603844,0.269228
13,0.346419,0.0,0.016624,0.267370,0.0,0.404097,0.435173,0.328430,0.326913,0.416607,...,0.284706,0.366839,0.323073,0.201631,0.317710,0.318264,0.363034,0.137204,0.243073,0.245607
49,0.784856,0.0,0.043533,0.660110,0.0,1.000000,0.788099,0.488756,0.831162,0.452173,...,0.700888,0.583539,0.375823,0.479971,0.467987,0.236863,0.861263,0.416288,0.547573,0.606196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,0.614315,0.0,0.044724,0.619341,0.0,0.629111,0.620820,0.306229,0.648028,0.365265,...,0.469999,0.575589,0.261394,0.607160,0.394985,0.083760,0.702964,0.430872,0.611824,0.364559
193,0.555652,0.0,0.039099,0.560815,0.0,0.590868,0.650392,0.354905,0.621517,0.418917,...,0.456738,0.569125,0.312587,0.473137,0.395865,0.161130,0.649355,0.343891,0.526937,0.345268
175,0.568726,0.0,0.045188,0.671287,0.0,0.595907,0.709485,0.397661,0.621674,0.424917,...,0.430299,0.614841,0.345628,0.585623,0.461028,0.169487,0.668554,0.495549,0.670250,0.292998


In [15]:
all_group_makers_recom = pd.DataFrame(columns = ['GroupId', 'Recom_MakersId', 'Recom_MakersScore'])

for i in (sorted(group_makers_recom_score.index)):
    temp = pd.DataFrame(columns = ['GroupId', 'Recom_MakersId', 'Recom_MakersScore'])
    temp['GroupId'] = [i] * len(group_makers_recom_score.columns)
    temp['Recom_MakersId'] = group_makers_recom_score.loc[i].sort_values(ascending=False).index
    temp['Recom_MakersScore'] = group_makers_recom_score.loc[i].sort_values(ascending=False).values
    all_group_makers_recom = pd.concat([all_group_makers_recom, temp], ignore_index=True)
all_group_makers_recom

,GroupId,Recom_MakersId,Recom_MakersScore
0,0,107,0.499401
1,0,121,0.498729
2,0,202,0.493632
3,0,70,0.486716
4,0,158,0.477845
...,...,...,...
24472,194,75,0.000000
24473,194,57,0.000000
24474,194,83,0.000000
24475,194,47,0.000000


In [16]:
all_group_makers_recom = all_group_makers_recom.set_index(['GroupId','Recom_MakersId','Recom_MakersScore'])
all_group_makers_recom

Empty DataFrame
Columns: []
Index: [(0, 107, 0.499400520709021), (0, 121, 0.4987293568300236), (0, 202, 0.4936321258809197), (0, 70, 0.4867156905351207), (0, 158, 0.4778451674094867), (0, 159, 0.4702453202099814), (0, 69, 0.4696719186611623), (0, 117, 0.4664018567797268), (0, 17, 0.4637007368025708), (0, 137, 0.4557307284232907), (0, 230, 0.4549056965704633), (0, 151, 0.4544981653950207), (0, 250, 0.4537711798054716), (0, 110, 0.4461535804053305), (0, 119, 0.4411947438708068), (0, 62, 0.4358197268901584), (0, 93, 0.4345395702330298), (0, 51, 0.4318253693319618), (0, 92, 0.4307307535802184), (0, 152, 0.43021148667202), (0, 238, 0.4297286576532788), (0, 48, 0.4269641324654337), (0, 179, 0.4217134616602154), (0, 120, 0.4215461743253271), (0, 1, 0.419611334116791), (0, 68, 0.4173094579954675), (0, 42, 0.4154915876207005), (0, 207, 0.4120531970476276), (0, 132, 0.405271582820662), (0, 191, 0.4035327291203245), (0, 15, 0.4032311352163485), (0, 99, 0.4013063155967479), (0, 22, 0.4009872607040953), (0, 199, 0.4002727051233418), (0, 141, 0.3938674720631849), (0, 52, 0.3921113048386754), (0, 231, 0.3915847796389494), (0, 136, 0.3913758059515785), (0, 247, 0.38041093381749), (0, 135, 0.3764950216610232), (0, 108, 0.3763654633712167), (0, 104, 0.3757615694296449), (0, 98, 0.3742451521263281), (0, 118, 0.3735345640161656), (0, 66, 0.3732362227340453), (0, 29, 0.3705260217946514), (0, 100, 0.370039675258095), (0, 165, 0.3684070606828604), (0, 211, 0.3684027837995514), (0, 37, 0.3682894394754829), (0, 74, 0.3675404861619678), (0, 113, 0.3662868776581821), (0, 156, 0.3619941420050115), (0, 21, 0.3606838717648736), (0, 91, 0.3595310525910931), (0, 252, 0.3591299128236018), (0, 164, 0.3587946118272623), (0, 224, 0.3533299803165103), (0, 97, 0.3513877421314569), (0, 50, 0.3497938582025367), (0, 133, 0.3468561088700447), (0, 169, 0.3456961280757163), (0, 16, 0.3429909035733696), (0, 181, 0.3423945109535716), (0, 210, 0.3393452701141347), (0, 194, 0.3389506891936164), (0, 219, 0.3371526253399283), (0, 111, 0.3336234724483301), (0, 166, 0.3312305953791933), (0, 105, 0.3287451514697765), (0, 123, 0.3286098386188591), (0, 253, 0.3282197002067809), (0, 172, 0.326571417741591), (0, 223, 0.3253279026047707), (0, 208, 0.3242320724322238), (0, 225, 0.3223436569763062), (0, 129, 0.3218309686638417), (0, 103, 0.3208644714637007), (0, 112, 0.3192524934865415), (0, 228, 0.3183570570325409), (0, 144, 0.3172238526363521), (0, 240, 0.3158069838459028), (0, 147, 0.3151111383670626), (0, 163, 0.3146153098951721), (0, 30, 0.3136494701953505), (0, 146, 0.3131939049115068), (0, 153, 0.3122475444929043), (0, 180, 0.3109409890608322), (0, 174, 0.3099496008498759), (0, 38, 0.3097756974470402), (0, 212, 0.3096992767183437), (0, 198, 0.3095135019937207), (0, 167, 0.3093672502627222), (0, 140, 0.308851933186141), (0, 46, 0.3084571766684688), (0, 41, 0.3072047546251554), (0, 138, 0.3060140619985354), (0, 173, 0.305910305676024), (0, 220, 0.3058979852207064), (0, 94, 0.3055104490807659), ...]

[24477 rows x 0 columns]

#### 실행 안 해도 됨

In [17]:
all_group_makers_recom.to_csv('data/recom_df/all_group_makers_recom.csv')

In [21]:
all_group_makers_recom = pd.read_csv('data/recom_df/all_group_makers_recom.csv')
all_group_makers_recom.set_index(['GroupId','Recom_MakersId','Recom_MakersScore'],inplace=True)
all_group_makers_recom

Empty DataFrame
Columns: []
Index: [(0, 107, 0.499400520709021), (0, 121, 0.4987293568300236), (0, 202, 0.4936321258809197), (0, 70, 0.4867156905351207), (0, 158, 0.4778451674094867), (0, 159, 0.4702453202099814), (0, 69, 0.4696719186611623), (0, 117, 0.4664018567797268), (0, 17, 0.4637007368025708), (0, 137, 0.4557307284232907), (0, 230, 0.4549056965704633), (0, 151, 0.4544981653950207), (0, 250, 0.4537711798054716), (0, 110, 0.4461535804053305), (0, 119, 0.4411947438708068), (0, 62, 0.4358197268901584), (0, 93, 0.4345395702330298), (0, 51, 0.4318253693319618), (0, 92, 0.4307307535802184), (0, 152, 0.43021148667202), (0, 238, 0.4297286576532788), (0, 48, 0.4269641324654337), (0, 179, 0.4217134616602154), (0, 120, 0.4215461743253271), (0, 1, 0.419611334116791), (0, 68, 0.4173094579954675), (0, 42, 0.4154915876207005), (0, 207, 0.4120531970476276), (0, 132, 0.405271582820662), (0, 191, 0.4035327291203245), (0, 15, 0.4032311352163485), (0, 99, 0.4013063155967479), (0, 22, 0.4009872607040953), (0, 199, 0.4002727051233418), (0, 141, 0.3938674720631849), (0, 52, 0.3921113048386754), (0, 231, 0.3915847796389494), (0, 136, 0.3913758059515785), (0, 247, 0.38041093381749), (0, 135, 0.3764950216610232), (0, 108, 0.3763654633712167), (0, 104, 0.3757615694296449), (0, 98, 0.3742451521263281), (0, 118, 0.3735345640161656), (0, 66, 0.3732362227340453), (0, 29, 0.3705260217946514), (0, 100, 0.370039675258095), (0, 165, 0.3684070606828604), (0, 211, 0.3684027837995514), (0, 37, 0.3682894394754829), (0, 74, 0.3675404861619678), (0, 113, 0.3662868776581821), (0, 156, 0.3619941420050115), (0, 21, 0.3606838717648736), (0, 91, 0.3595310525910931), (0, 252, 0.3591299128236018), (0, 164, 0.3587946118272623), (0, 224, 0.3533299803165103), (0, 97, 0.3513877421314569), (0, 50, 0.3497938582025367), (0, 133, 0.3468561088700447), (0, 169, 0.3456961280757163), (0, 16, 0.3429909035733696), (0, 181, 0.3423945109535716), (0, 210, 0.3393452701141347), (0, 194, 0.3389506891936164), (0, 219, 0.3371526253399283), (0, 111, 0.3336234724483301), (0, 166, 0.3312305953791933), (0, 105, 0.3287451514697765), (0, 123, 0.3286098386188591), (0, 253, 0.3282197002067809), (0, 172, 0.326571417741591), (0, 223, 0.3253279026047707), (0, 208, 0.3242320724322238), (0, 225, 0.3223436569763062), (0, 129, 0.3218309686638417), (0, 103, 0.3208644714637007), (0, 112, 0.3192524934865415), (0, 228, 0.3183570570325409), (0, 144, 0.3172238526363521), (0, 240, 0.3158069838459028), (0, 147, 0.3151111383670626), (0, 163, 0.3146153098951721), (0, 30, 0.3136494701953505), (0, 146, 0.3131939049115068), (0, 153, 0.3122475444929043), (0, 180, 0.3109409890608322), (0, 174, 0.3099496008498759), (0, 38, 0.3097756974470402), (0, 212, 0.3096992767183437), (0, 198, 0.3095135019937207), (0, 167, 0.3093672502627222), (0, 140, 0.308851933186141), (0, 46, 0.3084571766684688), (0, 41, 0.3072047546251554), (0, 138, 0.3060140619985354), (0, 173, 0.305910305676024), (0, 220, 0.3058979852207064), (0, 94, 0.3055104490807659), ...]

[24477 rows x 0 columns]

> GroupId, FoodId, FoodId_Score DF 만들기

In [200]:
# 유저를 담고 있는 set
users = set()
for u in group_makers_df['UserId'].unique():
    users.add(u)

In [201]:
i = 0
#행은 Userid, 열은 FoodId
user_food_recom_df = pd.DataFrame(index=[u for u in recently_n_food_dic], columns=[f for f in food_df['FoodId'].unique()])

# 사용자가 가장 최근에 먹은 음식 기반 음식 추천 #
def by_n_recently_ordered(foodid):
    global user_food_recom_df
    cnt = 0
    # 주어진 음식과 다른 음식의 similarity를 가져온다
    sim_scores = food_cs_df.loc[[f for f in foodid]].apply(lambda x: sum(x), axis = 0)
    sim_scores = sim_scores.apply(lambda x : x/len(foodid))
    
    for f_score in sim_scores.values:
        user_food_recom_df.iloc[i,cnt] = f_score
        cnt += 1

# 위에서 만든 user_food_recom_df value값 저장해주는 함수
for u in tqdm(users):
    by_n_recently_ordered(recently_n_food_dic[u])
    i += 1

100%|██████████| 4371/4371 [11:18<00:00,  6.45it/s]


In [203]:
user_food_recom_df.to_csv('data/recom_df/user_food_recom_df.csv')

In [241]:
            # 열은 Foodid
#행은 Userid
user_food_recom_score = pd.read_csv('data/recom_df/user_food_recom_df.csv')
user_food_recom_score.rename(columns={'Unnamed: 0' : 'Userid'}, inplace= True)
user_food_recom_score.set_index('Userid', inplace = True)
user_food_recom_score

,18,51,50,43,42,44,47,39,41,49,...,10068,9812,10794,10772,7729,9941,7731,10453,10773,10633
Userid,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.287799,0.407448,0.330219,0.393783,0.364432,0.364432,0.414123,0.100817,0.454578,0.292899
347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.366331,0.462961,0.258143,0.445426,0.218551,0.218551,0.288860,0.107605,0.445426,0.350559
352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.316088,0.393750,0.300950,0.460191,0.356458,0.356458,0.426694,0.068006,0.455745,0.230858
381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.280167,0.423532,0.335618,0.354461,0.230370,0.230370,0.261510,0.057886,0.406775,0.246440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.150402,0.685800,0.288461,0.572064,0.247023,0.247023,0.342505,0.142702,0.630242,0.141802
8166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.080582,0.696311,0.363636,0.545455,0.311400,0.311400,0.311400,0.150756,0.636364,0.090909
8175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.198675,0.566987,0.369201,0.507436,0.341010,0.341010,0.412621,0.092450,0.564324,0.196261


In [354]:
all_user_food_recom = pd.DataFrame(columns = ['Userid', 'Recom_Foodid', 'Recom_FoodScore'])

for i in tqdm((sorted(user_food_recom_score.index))):
    temp = pd.DataFrame(columns = ['Userid', 'Recom_Foodid', 'Recom_FoodScore'])
    temp['Userid'] = [i] * len(user_food_recom_score.columns)
    temp['Recom_Foodid'] = user_food_recom_score.loc[i].sort_values(ascending=False).index
    temp['Recom_FoodScore'] = user_food_recom_score.loc[i].sort_values(ascending=False).values
    all_user_food_recom = pd.concat([all_user_food_recom, temp], ignore_index=True)
all_user_food_recom

100%|██████████| 4371/4371 [16:50<00:00,  4.33it/s]


,Userid,Recom_Foodid,Recom_FoodScore
0,3,18,0.0
1,3,6713,0.0
2,3,6709,0.0
3,3,6733,0.0
4,3,6732,0.0
...,...,...,...
26982178,8274,742,0.0
26982179,8274,695,0.0
26982180,8274,697,0.0
26982181,8274,744,0.0


In [363]:
all_user_food_recom = all_user_food_recom.set_index(['Userid','Recom_Foodid','Recom_FoodScore'])
all_user_food_recom

Empty DataFrame
Columns: []
Index: [(3, 18, 0.0), (3, 6713, 0.0), (3, 6709, 0.0), (3, 6733, 0.0), (3, 6732, 0.0), (3, 5691, 0.0), (3, 6084, 0.0), (3, 5232, 0.0), (3, 6731, 0.0), (3, 6728, 0.0), (3, 6737, 0.0), (3, 4500, 0.0), (3, 6491, 0.0), (3, 6583, 0.0), (3, 6781, 0.0), (3, 6753, 0.0), (3, 6738, 0.0), (3, 6719, 0.0), (3, 5456, 0.0), (3, 6195, 0.0), (3, 6739, 0.0), (3, 6584, 0.0), (3, 6196, 0.0), (3, 6701, 0.0), (3, 6783, 0.0), (3, 6734, 0.0), (3, 6342, 0.0), (3, 6720, 0.0), (3, 6424, 0.0), (3, 6618, 0.0), (3, 6786, 0.0), (3, 6785, 0.0), (3, 6343, 0.0), (3, 5719, 0.0), (3, 5824, 0.0), (3, 6784, 0.0), (3, 6754, 0.0), (3, 6735, 0.0), (3, 4502, 0.0), (3, 6744, 0.0), (3, 6782, 0.0), (3, 6747, 0.0), (3, 6654, 0.0), (3, 6707, 0.0), (3, 6551, 0.0), (3, 6745, 0.0), (3, 6626, 0.0), (3, 6621, 0.0), (3, 3430, 0.0), (3, 5548, 0.0), (3, 6577, 0.0), (3, 6604, 0.0), (3, 6595, 0.0), (3, 5809, 0.0), (3, 6502, 0.0), (3, 6562, 0.0), (3, 4446, 0.0), (3, 6353, 0.0), (3, 6552, 0.0), (3, 6574, 0.0), (3, 6717, 0.0), (3, 6087, 0.0), (3, 6553, 0.0), (3, 6632, 0.0), (3, 6420, 0.0), (3, 6462, 0.0), (3, 6303, 0.0), (3, 6436, 0.0), (3, 6573, 0.0), (3, 5556, 0.0), (3, 6579, 0.0), (3, 5193, 0.0), (3, 6558, 0.0), (3, 6722, 0.0), (3, 6576, 0.0), (3, 6452, 0.0), (3, 6568, 0.0), (3, 6434, 0.0), (3, 6099, 0.0), (3, 6723, 0.0), (3, 6625, 0.0), (3, 6500, 0.0), (3, 5826, 0.0), (3, 6559, 0.0), (3, 6082, 0.0), (3, 6090, 0.0), (3, 6432, 0.0), (3, 6623, 0.0), (3, 6104, 0.0), (3, 6488, 0.0), (3, 5684, 0.0), (3, 6554, 0.0), (3, 6428, 0.0), (3, 6134, 0.0), (3, 6575, 0.0), (3, 6143, 0.0), (3, 6547, 0.0), (3, 6721, 0.0), (3, 4330, 0.0), (3, 6865, 0.0), ...]

[26982183 rows x 0 columns]

#### 실행 안 해도 됨

In [367]:
# all_user_food_recom.to_csv('data/recom_df/all_user_food_recom.csv')

In [31]:
all_user_food_recom = pd.read_csv('data/recom_df/all_user_food_recom.csv')
all_user_food_recom.set_index(['Userid','Recom_Foodid','Recom_FoodScore'],inplace=True)
all_user_food_recom

Empty DataFrame
Columns: []
Index: [(3, 18, 0.0), (3, 6713, 0.0), (3, 6709, 0.0), (3, 6733, 0.0), (3, 6732, 0.0), (3, 5691, 0.0), (3, 6084, 0.0), (3, 5232, 0.0), (3, 6731, 0.0), (3, 6728, 0.0), (3, 6737, 0.0), (3, 4500, 0.0), (3, 6491, 0.0), (3, 6583, 0.0), (3, 6781, 0.0), (3, 6753, 0.0), (3, 6738, 0.0), (3, 6719, 0.0), (3, 5456, 0.0), (3, 6195, 0.0), (3, 6739, 0.0), (3, 6584, 0.0), (3, 6196, 0.0), (3, 6701, 0.0), (3, 6783, 0.0), (3, 6734, 0.0), (3, 6342, 0.0), (3, 6720, 0.0), (3, 6424, 0.0), (3, 6618, 0.0), (3, 6786, 0.0), (3, 6785, 0.0), (3, 6343, 0.0), (3, 5719, 0.0), (3, 5824, 0.0), (3, 6784, 0.0), (3, 6754, 0.0), (3, 6735, 0.0), (3, 4502, 0.0), (3, 6744, 0.0), (3, 6782, 0.0), (3, 6747, 0.0), (3, 6654, 0.0), (3, 6707, 0.0), (3, 6551, 0.0), (3, 6745, 0.0), (3, 6626, 0.0), (3, 6621, 0.0), (3, 3430, 0.0), (3, 5548, 0.0), (3, 6577, 0.0), (3, 6604, 0.0), (3, 6595, 0.0), (3, 5809, 0.0), (3, 6502, 0.0), (3, 6562, 0.0), (3, 4446, 0.0), (3, 6353, 0.0), (3, 6552, 0.0), (3, 6574, 0.0), (3, 6717, 0.0), (3, 6087, 0.0), (3, 6553, 0.0), (3, 6632, 0.0), (3, 6420, 0.0), (3, 6462, 0.0), (3, 6303, 0.0), (3, 6436, 0.0), (3, 6573, 0.0), (3, 5556, 0.0), (3, 6579, 0.0), (3, 5193, 0.0), (3, 6558, 0.0), (3, 6722, 0.0), (3, 6576, 0.0), (3, 6452, 0.0), (3, 6568, 0.0), (3, 6434, 0.0), (3, 6099, 0.0), (3, 6723, 0.0), (3, 6625, 0.0), (3, 6500, 0.0), (3, 5826, 0.0), (3, 6559, 0.0), (3, 6082, 0.0), (3, 6090, 0.0), (3, 6432, 0.0), (3, 6623, 0.0), (3, 6104, 0.0), (3, 6488, 0.0), (3, 5684, 0.0), (3, 6554, 0.0), (3, 6428, 0.0), (3, 6134, 0.0), (3, 6575, 0.0), (3, 6143, 0.0), (3, 6547, 0.0), (3, 6721, 0.0), (3, 4330, 0.0), (3, 6865, 0.0), ...]

[26982183 rows x 0 columns]

In [56]:
# 8274 유저가 실제로 먹은 음식은
# recently_n_food_dic[8274] '된장찌개 + 조미김'
# 추천된 음식 6268 '사골우거지탕*'
all_user_food_recom.loc[8274]

,
Recom_Foodid,Recom_FoodScore
6268,0.631455
6269,0.631455
4342,0.611150
4343,0.611150
4404,0.605662
...,...
742,0.000000
695,0.000000
697,0.000000


# EXTRA. input을 통한 CB추천
### 그룹 35 쳐보기

In [20]:
# 데이터 프레임 생성
group_makers_recom_df = pd.DataFrame(
    columns=['GroupId','MakersId','MakersScore']
)
user_food_recom_df = pd.DataFrame(
    columns=['UserId','FoodId','FoodScore']
)
user_foodname_recom_df = pd.DataFrame(
    columns=['UserId', 'Eaten_FoodName','Recom_FoodName', 'FoodScore']
)


# 사용자가 가장 최근에 먹은 음식 기반 추천된 메이커스에서 판매하는 음식 추천 #
def by_recently_ordered(userid, n_of_recomm, makers):
    num = 1
    recom_food_score_list = []
    before_food = []
    foodid = recently_eaten_dic[userid]
    cs_include_food = set({})
    global user_food_recom_df, user_foodname_recom_df
    
    # 주어진 음식과 다른 음식의 similarity를 가져온다
    sim_scores = food_cs_df.loc[foodid]
    del sim_scores[f'{foodid}']
    makers_own_food_cs = sim_scores
    
    for f in [str(food) for food in makers_foods_dic[makers]]:
        if f in makers_own_food_cs.index:
            cs_include_food.add(f)
    makers_own_food_cs = makers_own_food_cs.loc[[f for f in cs_include_food]].sort_values(ascending=False)
    
    recom_food_score_list.append(makers_own_food_cs.index)
    recom_food_score_list.append(makers_own_food_cs.values)
    
    print(f"사용자{userid} - 최근에 먹은 음식 {food_name_dic[foodid]}({foodid}) 바탕으로 추천된 음식 Top{n_of_recomm}\n")
    
    for i in range(len(recom_food_score_list[0])):
        if food_name_dic[int(recom_food_score_list[0][i])] in before_food: # 이전에 먹은 음식이면 pass
            pass
        elif num > n_of_recomm:
            break
        else:
            print(f'Top{num} 음식: {food_name_dic[int(recom_food_score_list[0][i])]}({recom_food_score_list[0][i]}), 점수: {recom_food_score_list[1][i]}\n')
            before_food.append(food_name_dic[int(recom_food_score_list[0][i])]) # 먹은 음식 list에 추가
            # 유저-음식 추천 데이터프레임에 저장
            user_food_recom_df = user_food_recom_df.append({'UserId' : userid, 'FoodId' : recom_food_score_list[0][i], 'FoodScore' : recom_food_score_list[1][i]}, ignore_index=True)
            user_foodname_recom_df = user_foodname_recom_df.append({'UserId' : userid, 'Eaten_FoodName' : food_name_dic[foodid], 'Recom_FoodName' : food_name_dic[int(recom_food_score_list[0][i])], 'FoodScore' : recom_food_score_list[1][i]}, ignore_index=True)
            num += 1

# 그룹이 가장 최근 이용한 메이커스 기반 메이커스 추천 #
def by_recently_used(makersid, n_of_recomm):
    recom_makers_score_list = []
    num = 0
    # 주어진 메이커스와 다른 메이커스의 similarity를 가져온다
    sim_scores = makers_cs_df.loc[makersid]
    sim_scores = sim_scores.sort_values(ascending = False)[1:n_of_recomm + 1]
    
    recom_makers_score_list.append(sim_scores.index)
    recom_makers_score_list.append(sim_scores.values)
    
    for i in range(len(recom_makers_score_list[0])):
        num += 1
        print(f"\n\n그룹{groupid}이(가) 최근에 이용한 메이커스{makersid} 바탕으로 추천된 메이커스 Top{num}")
        print(f'Top{num} 메이커스: {recom_makers_score_list[0][i]}, 점수: {recom_makers_score_list[1][i]}\n')
        # 그룹-메이커스 추천 데이터프레임에 저장
        group_makers_recom_df.loc[i] = [groupid, recom_makers_score_list[0][i], recom_makers_score_list[1][i]]
        makers = recom_makers_score_list[0][i]
    
        for userid in group_users_dic[groupid]:
            num_recomm_food = 3 # 추천 받을 음식 갯수
            # num_recomm_food = int(input()) # 추천 받을 음식 갯수
            by_recently_ordered(userid, num_recomm_food, makers)
            
groupid = int(input("그룹 아이디(GroupId)를 입력 : "))
top = int(input("추천받을 메이커스 개수 입력 : "))
by_recently_used(recently_makers_dic[groupid], top)



그룹35이(가) 최근에 이용한 메이커스94 바탕으로 추천된 메이커스 Top1
Top1 메이커스: 46, 점수: 0.9613409059110354

사용자1643 - 최근에 먹은 음식 차돌된장찌개 도시락*(2142) 바탕으로 추천된 음식 Top3

Top1 음식: 갈비탕*(2070), 점수: 0.9166666666666669

Top2 음식: 순대국밥*(2069), 점수: 0.75

Top3 음식: 소불고기백반*(1591), 점수: 0.5222329678670934

사용자1588 - 최근에 먹은 음식 참치김치찌개 도시락*(2126) 바탕으로 추천된 음식 Top3

Top1 음식: 순대국밥*(2069), 점수: 0.7302967433402214

Top2 음식: 갈비탕*(2070), 점수: 0.7302967433402214

Top3 음식: 주꾸미비빔밥*(2080), 점수: 0.5477225575051661

사용자1591 - 최근에 먹은 음식 돼지갈비 도시락*(2129) 바탕으로 추천된 음식 Top3

Top1 음식: 대패삼겹구이*(1529), 점수: 0.9166666666666669

Top2 음식: 제육백반*(1835), 점수: 0.8198915917499228

Top3 음식: 소불고기백반*(1591), 점수: 0.6963106238227912

사용자1595 - 최근에 먹은 음식 잠봉 샌드위치*(1766) 바탕으로 추천된 음식 Top3

Top1 음식: 대패삼겹구이*(1529), 점수: 0.3481553119113956

Top2 음식: 순대국밥*(2069), 점수: 0.3481553119113956

Top3 음식: 제육백반*(1835), 점수: 0.3113995776646091

사용자1597 - 최근에 먹은 음식 참치김치찌개*(881) 바탕으로 추천된 음식 Top3

Top1 음식: 순대국밥*(2069), 점수: 0.7833494518006401

Top2 음식: 갈비탕*(2070), 점수: 0.7833494518006401

Top3 음식: 

/var/folders/8p/0rn2lvdj485dhnwbsnpgnb380000gp/T/ipykernel_1147/2264052644.py:56: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  sim_scores = sim_scores.sort_values(ascending = False)[1:n_of_recomm + 1]
/var/folders/8p/0rn2lvdj485dhnwbsnpgnb380000gp/T/ipykernel_1147/2264052644.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  user_food_recom_df = user_food_recom_df.append({'UserId' : userid, 'FoodId' : recom_food_score_list[0][i], 'FoodScore' : recom_food_score_list[1][i]}, ignore_index=True)
/var/folders/8p/0rn2lvdj485dhnwbsnpgnb380000gp/T/ipykernel_1147/2264052644.py:47: FutureWarning: The frame.append method is deprecated and will be remove

In [ ]:
group_makers_recom_df # 그룹 메이커스 추천

,GroupId,MakersId,MakersScore
0,35.0,46.0,0.961341


In [ ]:
print(len(group_users_dic[35])) # 그룹 35의 유저는 6명

6


In [ ]:
user_food_recom_df.set_index(['UserId','FoodId']) # 유저당 음식 3개 추천이므로 index는 17까지

FoodScore
UserId FoodId           
1643   1592     0.916667
       1677     0.750000
       1591     0.522233
1588   2070     0.730297
       1677     0.730297
       1836     0.547723
1591   1529     0.916667
       2083     0.819892
       1591     0.696311
1595   1529     0.348155
       1677     0.348155
       2083     0.311400
1597   2070     0.783349
       1677     0.783349
       1836     0.522233
1598   1591     0.545455
       1835     0.467099
       1592     0.435194
1643   1869     0.916667
       1961     0.870388
       1865     0.833333
1588   1865     0.821584
       1863     0.789352
       1963     0.789352
1591   1964     0.696311
       1863     0.640513
       1974     0.609272
1595   1964     0.363636
       1863     0.334497
       1974     0.181818
1597   1863     0.836242
       1963     0.836242
       1865     0.783349
1598   1965     0.727273
       1962     0.696311
       1974     0.636364

In [ ]:
# 음식 이름으로 확인
user_foodname_recom_df.set_index(['UserId','Eaten_FoodName', 'Recom_FoodName'])

FoodScore
UserId Eaten_FoodName Recom_FoodName           
1643   차돌된장찌개 도시락*    갈비탕*             0.916667
                      순대국밥*            0.750000
                      소불고기백반*          0.522233
1588   참치김치찌개 도시락*    갈비탕*             0.730297
                      순대국밥*            0.730297
                      주꾸미비빔밥*          0.547723
1591   돼지갈비 도시락*      대패삼겹구이*          0.916667
                      제육백반*            0.819892
                      소불고기백반*          0.696311
1595   잠봉 샌드위치*       대패삼겹구이*          0.348155
                      순대국밥*            0.348155
                      제육백반*            0.311400
1597   참치김치찌개*        갈비탕*             0.783349
                      순대국밥*            0.783349
                      주꾸미비빔밥*          0.522233
1598   불고기 곡물 필라프*    소불고기백반*          0.545455
                      제육백반*            0.467099
                      갈비탕*             0.435194
1643   차돌된장찌개 도시락*    소고기 된장찌개*        0.916667
                      진곰탕반상*           0.870388
                      육개장*             0.833333
1588   참치김치찌개 도시락*    육개장*             0.821584
                      돼지고기 김치찌개*       0.789352
                      해물순두부찌개*         0.789352
1591   돼지갈비 도시락*      제육덮밥*            0.696311
                      돼지고기 김치찌개*       0.640513
                      뚝배기불고기반상*        0.609272
1595   잠봉 샌드위치*       제육덮밥*            0.363636
                      돼지고기 김치찌개*       0.334497
                      뚝배기불고기반상*        0.181818
1597   참치김치찌개*        돼지고기 김치찌개*       0.836242
                      해물순두부찌개*         0.836242
                      육개장*             0.783349
1598   불고기 곡물 필라프*    불고기 덮밥*          0.727273
                      오색비빔밥*           0.696311
                      뚝배기불고기반상*        0.636364

In [ ]:
# 그룹 35이 가장 최근에 이용한 메이커스 확인
recently_makers_dic[35] # 가장 최근에 이용한 메이커스는 94

94

In [ ]:
# 메이커스 간 유사도 확인 : 메이커스 추천에 사용
makers_cs_df.loc[94].sort_values(ascending=False) # 94을 이용해서 46이 추천됨

MakersId
94     1.000000
46     0.961341
76     0.925787
219    0.909059
146    0.905949
         ...   
72     0.000000
35     0.000000
47     0.000000
75     0.000000
20     0.000000
Name: 94, Length: 199, dtype: float64

In [ ]:
# 35그룹의 유저 확인
group_users_dic[35]

{1588, 1591, 1595, 1597, 1598, 1643}

In [ ]:
# 35 그룹의 1643유저가 가장 최근에 먹은 음식id 확인
recently_eaten_dic[1643] # 유저 1643이 먹은 가장 최근 음식id는 2142

2142

In [ ]:
# 추천된 makers에서 판매하는 음식 확인
for food in makers_foods_dic[46]:
    print(food)

131
132
1029
1675
1676
1677
2069
2070
285
286
2080
2081
2083
1835
1836
429
430
1589
1591
568
1592
187
188
575
449
450
203
1240
217
1241
1242
737
738
1639
236
237
1010
1529
1530


In [ ]:
print(1592 in makers_foods_dic[46]) # 추천된 메이커스 46에서 1592아이템을 판매함
print(2070 in makers_foods_dic[46]) # 추천된 메이커스 46에서 2070아이템을 판매함
print(2081 in makers_foods_dic[46]) # 추천된 메이커스 46에서 2081아이템을 판매함

True
True
True


In [ ]:
# 추천된 메이커스에서 판매하는 음식들에 대한 순위 확인

recom_food_score_list = [1592, 2070, 2081]
cs_include_food = set({})

# 주어진 음식과 다른 음식의 similarity를 가져온다
sim_scores = food_cs_df.loc[2142]
del sim_scores[f'{2142}']
makers_own_food_cs = sim_scores

for f in [str(food) for food in makers_foods_dic[46]]:
    if f in makers_own_food_cs.index:
        cs_include_food.add(f)
makers_own_food_cs = makers_own_food_cs.loc[[f for f in cs_include_food]].sort_values(ascending=False)

# 추천된 음식들 점수
makers_own_food_cs

1592    0.916667
2081    0.916667
2070    0.916667
1677    0.750000
2069    0.750000
1591    0.522233
1529    0.416667
1029    0.416667
1836    0.416667
1676    0.416667
2080    0.416667
2083    0.372678
1240    0.372678
1241    0.372678
1835    0.372678
188     0.000000
450     0.000000
236     0.000000
217     0.000000
429     0.000000
187     0.000000
1639    0.000000
203     0.000000
738     0.000000
1242    0.000000
449     0.000000
1010    0.000000
575     0.000000
132     0.000000
430     0.000000
568     0.000000
1530    0.000000
1589    0.000000
286     0.000000
131     0.000000
237     0.000000
1675    0.000000
285     0.000000
737     0.000000
Name: 2142, dtype: float64

크롤링 한 음식이 저희가 받은 데이터에 존재하지 않았어서 cs 매트릭스에서 해당 음식과 다른 유사도를 참조하는데 오류 발생
0으로 해줘야할지..
> 일단 없는 거는 제외해서 처리를 해두었음